Library Import and Pandas Options

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import date_range
import numpy as np


# Show all columns
pd.set_option('display.max_columns', 100)
# Show all rows (if needed, or set a limit)
pd.set_option('display.max_rows', 100)
# Disable line wrapping (prevent splitting columns into multiple lines)
pd.set_option('display.width', 100)
# Increase column width so data is not cut off
pd.set_option('display.max_colwidth', 50)

Data Loading

In [ ]:
# Load Sheet from Excel
all_sheets = pd.read_excel("2024-12-09_ATM_Branch_Data.xlsx", sheet_name=None)

# Define sheet names
atm =all_sheets['ATM']
events_holidays=all_sheets['Events_Holidays']


Adding Active Days

In [ ]:
# Adding active flag to ATMs
def add_is_active_flag(atm_df: pd.DataFrame,
                       id_col: str = "CASHP_ID",
                       date_col: str = "DATE",
                       y_col: str = "WITHDRWLS",
                       flag_name: str = "is_active") -> pd.DataFrame:
    def _mark(group):
        group = group.sort_values(date_col)
        group[flag_name] = (group[y_col].gt(0)).cumsum().clip(upper=1).astype(int)
        return group

    return atm_df.groupby(id_col, group_keys=False).apply(_mark)

atm = add_is_active_flag(atm)

Statistical Analysis of ATM Data

In [ ]:
# ATM
atm_stats = atm.groupby('CASHP_ID')['WITHDRWLS'].agg(['count', 'sum', 'mean', 'median', 'std', 'min', 'max']).reset_index()
atm_stats.columns = ['CASHP_ID', 'Count', 'Total_WITHDRWLS', 'Mean', 'Median', 'Std_Dev', 'Min', 'Max']
print(atm_stats)

Single ATM Averages

In [ ]:
# Grouping (WITH zeros)
atm_grouped_with_zeros = (
    atm.groupby('CASHP_ID')['WITHDRWLS']
    .mean()
    .reset_index()
    .rename(columns={'WITHDRWLS': 'WITH_zeros'})
)

# Grouping (WITHOUT zeros)
atm_grouped_without_zeros = (
    atm.loc[atm['WITHDRWLS'] > 0]
    .groupby('CASHP_ID')['WITHDRWLS']
    .mean()
    .reset_index()
    .rename(columns={'WITHDRWLS': 'WITHOUT_zeros'})
)

# Plot 1: Average ATM Withdrawals (WITH zeros)
plt.figure(figsize=(12, 6))
sns.barplot(data=atm_grouped_with_zeros, x='CASHP_ID', y='WITH_zeros', palette='Blues_d')
plt.title('Average ATM Withdrawals per CASHP_ID (WITH zeros)')
plt.xlabel('CASHP_ID')
plt.ylabel('Average Withdrawal')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("avg_withdrawals_atm_WITH_zeros.png", dpi=600, bbox_inches="tight")
plt.show()

# Plot 2: Average ATM Withdrawals (WITHOUT zeros)
plt.figure(figsize=(12, 6))
sns.barplot(data=atm_grouped_without_zeros, x='CASHP_ID', y='WITHOUT_zeros', palette='Purples_d')
plt.title('Average ATM Withdrawals per CASHP_ID (WITHOUT zeros)')
plt.xlabel('CASHP_ID')
plt.ylabel('Average Withdrawal')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("avg_withdrawals_atm_WITHOUT_zeros.png", dpi=600, bbox_inches="tight")
plt.show()




Holiday Addition to Data

In [ ]:
# Special Day means holidays in the year
special_days = [
    '2006-01-01', '2006-01-10', '2006-01-11', '2006-01-12', '2006-01-13',
    '2006-04-23', '2006-05-19', '2006-08-30', '2006-10-23', '2006-10-24', '2006-10-25',
    '2006-10-29', '2006-12-31',

    '2007-01-01', '2007-01-02', '2007-01-03', '2007-04-23', '2007-05-19',
    '2007-08-30', '2007-10-12', '2007-10-13', '2007-10-14', '2007-10-29',
    '2007-12-20', '2007-12-21', '2007-12-22', '2007-12-23', '2007-12-31',

    '2008-01-01', '2008-04-23', '2008-05-19', '2008-08-30',
    '2008-09-30', '2008-10-01', '2008-10-02', '2008-10-29',
    '2008-12-08', '2008-12-09', '2008-12-10', '2008-12-11', '2008-12-31'
]

school_holidays = list(date_range('2006-01-16', '2006-01-29', freq='D')) + \
                  list(date_range('2006-06-12', '2006-09-11', freq='D')) + \
                  list(date_range('2007-01-29', '2007-02-09', freq='D')) + \
                  list(date_range('2007-06-19', '2007-09-17', freq='D')) + \
                  list(date_range('2008-01-29', '2008-02-08', freq='D')) + \
                  list(date_range('2008-06-16', '2008-09-12', freq='D'))

school_holidays = [d.strftime('%Y-%m-%d') for d in school_holidays]

# These are half working days.
half_days = [
    '2006-01-09', '2006-10-22', '2006-12-30',
    '2007-10-11', '2007-12-19',
    '2008-09-29', '2008-12-07'
]

df_special = pd.DataFrame({'date': special_days, 'type': 'official_holiday'})
df_school = pd.DataFrame({'date': school_holidays, 'type': 'school_holiday'})
df_half = pd.DataFrame({'date': half_days, 'type': 'half_day'})

custom_events = pd.concat([df_special, df_school, df_half], ignore_index=True)
custom_events['date'] = pd.to_datetime(custom_events['date'])
custom_events = custom_events.drop_duplicates()

Average Withdrawals in Holidays, Half Days and Normal Days

In [ ]:
atm['DATE'] = pd.to_datetime(atm['DATE'])
atm_merged = atm.merge(custom_events, left_on='DATE', right_on='date', how='left')
atm_merged['day_type'] = atm_merged['type'].fillna('normal_day')

# Exclude zero-withdrawal days
atm_merged = atm_merged[atm_merged['WITHDRWLS'] > 0]

day_type_summary = atm_merged.groupby('day_type')['WITHDRWLS'].agg(['mean', 'median', 'std', 'count']).reset_index()
print(day_type_summary)

plt.figure(figsize=(10, 6))
sns.barplot(data=day_type_summary, x='day_type', y='mean', palette='Set2')
plt.title('Average Withdrawals: Holiday vs. Normal Day')
plt.xlabel('Day Type')
plt.ylabel('Average Withdrawals')
plt.xticks(rotation=30)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save PNG
plt.savefig("avg_withdrawals_day_type.png", dpi=600, bbox_inches="tight")
plt.show()

Weekday Averages

In [ ]:
# Exclude zero-withdrawal days
atm_nz = atm[atm['WITHDRWLS'] > 0].copy()

# Weekday summary on non-zero days
atm_nz['weekday'] = atm_nz['DATE'].dt.day_name()
weekday_summary = atm_nz.groupby('weekday')['WITHDRWLS'].agg(['mean', 'median', 'std', 'count']).reset_index()

weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_summary['weekday'] = pd.Categorical(weekday_summary['weekday'], categories=weekday_order, ordered=True)
weekday_summary = weekday_summary.sort_values('weekday')

print(weekday_summary)

sns.barplot(data=weekday_summary, x='weekday', y='mean')
plt.title("average withdrawal (non-zero days)")
plt.ylabel("mean amount")
plt.xlabel("Day")
plt.xticks(rotation=45)
plt.tight_layout()

# Save PNG
plt.savefig("avg_withdrawals_by_weekday_nozeros.png", dpi=600, bbox_inches="tight")
plt.show()


Month Day Averages

In [ ]:
# Convert to datetime and exclude zero-withdrawal days
atm['DATE'] = pd.to_datetime(atm['DATE'], errors='coerce')
atm_nz = atm[atm['WITHDRWLS'] > 0].copy()

# Day-of-month summary (non-zero days only)
atm_nz['day_of_month'] = atm_nz['DATE'].dt.day
day_summary = (
    atm_nz.groupby('day_of_month')['WITHDRWLS']
    .agg(['mean', 'median', 'std', 'count'])
    .reset_index()
)

# Plot + PNG export
plt.figure(figsize=(14, 6))
sns.barplot(
    data=day_summary,
    x='day_of_month',
    y='mean',
    order=list(range(1, 32)),
    palette='viridis'
)
plt.title('Average Withdrawals by Day of the Month (Non-zero Days)')
plt.xlabel('Day of Month (1–31)')
plt.ylabel('Average Withdrawals')
plt.xticks(range(1, 32))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig("avg_withdrawals_by_day_of_month_nozeros.png", dpi=600, bbox_inches="tight")
plt.show()


Active Period Analysis

In [ ]:
atm_merged['is_weekend'] = atm_merged['DATE'].dt.weekday >= 5
atm_merged['is_special_day'] = atm_merged['day_type'] != 'normal_day'
atm_merged['is_non_working_day'] = atm_merged['is_weekend'] | atm_merged['is_special_day']

# Subset only non-working days
non_working_days = atm_merged[atm_merged['is_non_working_day']]

# Find ATMs with total withdrawals = 0 on ALL non-working days
no_activity_atms = (
    non_working_days.groupby("CASHP_ID")["WITHDRWLS"]
    .sum()
    .reset_index()
)
no_activity_atms = no_activity_atms[no_activity_atms["WITHDRWLS"] == 0]

print("Number of ATMs with no activity on any non-working day:", no_activity_atms.shape[0])
print("ATM IDs:\n", no_activity_atms["CASHP_ID"].tolist())

# merge with full ATM list to show active vs inactive
all_atms = atm_merged["CASHP_ID"].unique()
activity_status = pd.DataFrame({'CASHP_ID': all_atms})
activity_status['inactive_on_non_working_days'] = activity_status['CASHP_ID'].isin(no_activity_atms['CASHP_ID'])

# Visualize count
plt.figure(figsize=(6, 4))
sns.countplot(data=activity_status, x='inactive_on_non_working_days', palette='Set2')
plt.title('ATM Activity on Non-working Days')
plt.xlabel('Inactive on Weekends/Holidays?')
plt.ylabel('Number of ATMs')
plt.xticks([0, 1], ['Active', 'Inactive'])
plt.tight_layout()
plt.show()

Monthly Averages

In [ ]:
# Exclude zero-withdrawal days
atm_nz = atm[atm['WITHDRWLS'] > 0].copy()

# Extract month
atm_nz['month'] = atm_nz['DATE'].dt.month

# Group by month
month_summary = (
    atm_nz.groupby('month')['WITHDRWLS']
    .agg(['mean', 'median', 'std', 'count'])
    .reset_index()
)

# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(data=month_summary, x='month', y='mean', palette='coolwarm')
plt.title('Average Withdrawals by Month (Non-zero Days)')
plt.xlabel('Month')
plt.ylabel('Average Withdrawals')
plt.xticks(
    range(0, 12),
    ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig("avg_withdrawals_by_month_nozeros.png", dpi=600, bbox_inches="tight")
plt.show()


Clustering Visualization

In [ ]:
# KMeans clustering with barplot visualization (is_active == 1 only)
from sklearn.cluster import KMeans

# Build ATM profiles (avg withdrawals only on active days)
atm_active = atm[atm["is_active"] == 1].copy()
atm_profiles = (
    atm_active.groupby("CASHP_ID")["WITHDRWLS"]
    .mean()
    .reset_index()
    .rename(columns={"WITHDRWLS": "avg_withdrawals"})
)

# Run clustering for k = 2, 3, 4
for k in (2, 3, 4):
    X = atm_profiles[["avg_withdrawals"]].values
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    atm_profiles[f"cluster_{k}"] = kmeans.fit_predict(X)

    # Sort by avg_withdrawals for nicer plotting
    plot_df = atm_profiles.sort_values("avg_withdrawals").copy()

    # Barplot visualization
    plt.figure(figsize=(14, 6))
    sns.barplot(
        data=plot_df,
        x="CASHP_ID",
        y="avg_withdrawals",
        hue=f"cluster_{k}",
        dodge=False,
        palette="tab10"
    )
    plt.title(f"KMeans clustering on avg_withdrawals (is_active=1) — k={k}")
    plt.xlabel("ATM ID")
    plt.ylabel("Average Withdrawals")
    plt.xticks(rotation=90)
    plt.legend(title=f"Cluster {k}")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()

    out_png = f"kmeans_active_avg_k{k}_bar.png"
    plt.savefig(out_png, dpi=600, bbox_inches="tight")
    plt.show()
    print(f"Saved {out_png}")
